In [1]:
import os
import re
from typing import Dict, Tuple
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras

import keras.layers as l
from keras import models, callbacks, utils, losses

In [27]:
text = ''
with open('medium_articles.txt', 'r', encoding='utf-8') as file:
    text = file.read(250000)

def get_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^a-zA-Z0-9 ,-]', '', s).strip() for s in text.split('.')]))
alp = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alp)}
index_word = {i: char for i, char in enumerate(alp)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

In [28]:
model = keras.Sequential([
    l.Embedding(len(alp), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.Bidirectional(l.LSTM(150, return_sequences=True)),
    l.Dropout(0.2),
    l.LSTM(512, return_sequences=True, stateful=True),
    l.Dense(len(alp) / 2, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    l.Dense(len(alp), activation='softmax')
])

In [29]:
model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=45, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/45
39/39 [==============================] - 45s 1s/step - loss: 10.6922 - accuracy: 0.0319
Epoch 2/45
39/39 [==============================] - 42s 1s/step - loss: 7.6535 - accuracy: 0.0316
Epoch 3/45
39/39 [==============================] - 37s 948ms/step - loss: 7.5274 - accuracy: 0.0341
Epoch 4/45
39/39 [==============================] - 37s 953ms/step - loss: 7.4704 - accuracy: 0.0346
Epoch 5/45
39/39 [==============================] - 37s 949ms/step - loss: 7.4612 - accuracy: 0.0320
Epoch 6/45
39/39 [==============================] - 37s 946ms/step - loss: 7.4296 - accuracy: 0.0330
Epoch 7/45
39/39 [==============================] - 37s 945ms/step - loss: 7.3848 - accuracy: 0.0338
Epoch 8/45
39/39 [==============================] - 37s 943ms/step - loss: 7.2141 - accuracy: 0.0330
Epoch 9/45
39/39 [==============================] - 39s 1s/step - loss: 7.0002 - accuracy: 0.0342
Epoch 10/45
39/39 [==============================] - 41s 1s/step - loss: 6.7717 - accuracy: 0.0320


In [30]:
model.fit(data, epochs=65, initial_epoch=45, verbose=1, steps_per_epoch=len(sequences) // BATCH_SIZE)

Epoch 46/65
39/39 [==============================] - 39s 995ms/step - loss: 3.3342 - accuracy: 0.3890
Epoch 47/65
39/39 [==============================] - 38s 964ms/step - loss: 3.2939 - accuracy: 0.3893
Epoch 48/65
39/39 [==============================] - 46s 1s/step - loss: 3.2845 - accuracy: 0.3857
Epoch 49/65
39/39 [==============================] - 44s 1s/step - loss: 3.2459 - accuracy: 0.3961
Epoch 50/65
39/39 [==============================] - 43s 1s/step - loss: 3.1997 - accuracy: 0.4042
Epoch 51/65
39/39 [==============================] - 41s 1s/step - loss: 3.1909 - accuracy: 0.4015
Epoch 52/65
39/39 [==============================] - 41s 1s/step - loss: 3.1349 - accuracy: 0.4100
Epoch 53/65
39/39 [==============================] - 42s 1s/step - loss: 3.1052 - accuracy: 0.4167
Epoch 54/65
39/39 [==============================] - 45s 1s/step - loss: 3.0833 - accuracy: 0.4144
Epoch 55/65
39/39 [==============================] - 43s 1s/step - loss: 3.0557 - accuracy: 0.4190
Epoc

In [32]:
def gen_next(sample, model, tokenizer, vocabulary, n_next, temperature, batch_size, word):
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [37]:
print(gen_next(
    sample='Where',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.4,
    batch_size=BATCH_SIZE,
    word=True
))

Where quietly significantly true falls discriminating whenever acquisitions Donald honestly necessarily phenomena Finally, ScienceCan attacking doctor initiatives Back phrases whisper SME


In [50]:
print(gen_next(
    sample='Who',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.2,
    batch_size=BATCH_SIZE,
    word=True
))

Who happy tell trail controversies, netcnnLiege-v-Benfica01 papers, plain CCPS, diagnoses paywall achieving personally Jones wore tying urgency Waste reference quarantining severe


In [58]:
print(gen_next(
    sample='Face',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

Face define drilling focuses premier June offline Cantt fight thrived copywriter brought govern traced Fantastic thicker chi complex Back components, Development,
